In [8]:
import numpy as np

opcodeInput=np.loadtxt("input_day17.txt",delimiter=",",dtype=np.int64)

class OpCodeManager:
    def __init__(self, opcode):
        self.opcode = opcode
        self.relativeBaseIdx=0
    def __getitem__(self, idx):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        return self.opcode[idx]
    def __setitem__(self, idx, value):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        self.opcode[idx] = value

def decodeCommand(command):
    code=command%100
    p1Mode=int(command/100)%10
    p2Mode=int(command/1000)%10
    p3Mode=int(command/10000)%10
    return [code, p1Mode, p2Mode, p3Mode]

def getArgument(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return opcode[parameter]
    elif mode == 1:
        # immediate mode
        return parameter
    else:
        # relative mode
        return opcode[opcode.relativeBaseIdx+parameter]
    
def getWritePosition(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return parameter
    elif mode == 1:
        # immediate mode
        print("ERROR! Immediate mode not intended for writing")
        print(mode, parameter, opcode)
    else:
        # relative mode
        return opcode.relativeBaseIdx+parameter

In [62]:
def getPositionOffsetFromCommand(command):
    if command == 1:
        # north
        return [0,-1]
    elif command == 2:
        # south
        return [0,1]
    elif command == 3:
        # west
        return [-1,0]
    elif command == 4:
        # east
        return [1,0]
    else:
        return [0,0]
    
def getNeighborhoodStatus(position, locationsMap):
    neighborhoodStatus = []
    for i in range(1,5):
        neighborhoodPos = tuple(position+getPositionOffsetFromCommand(i))
        if neighborhoodPos in locationsMap:
            neighborhoodStatus.append(locationsMap[neighborhoodPos])
        else:
             neighborhoodStatus.append(-1);
    return neighborhoodStatus

def getNeighborhoodStatusTuple(position, locationsMap):
    neighborhoodStatus = []
    for i in range(1,5):
        neighborhoodPos = tuple(position+getPositionOffsetFromCommand(i))
        if neighborhoodPos in locationsMap:
            neighborhoodStatus.append((neighborhoodPos,locationsMap[neighborhoodPos]))
        else:
             neighborhoodStatus.append((neighborhoodPos,-1));
    return neighborhoodStatus

class Computer:
    def __init__(self, opcodeMgr):
        self.opcode = opcodeMgr
        self.outputList=[]
        self.input=0
        
    def clearOutput(self):
        self.outputList=[]
    
    def visualize(self):
        outputStr=''
        for elem in self.outputList:
            outputStr += chr(elem)
        return outputStr
    
    def printOutputList(self):
        outputStr=''
        for elem in self.outputList:
            outputStr += chr(elem)
        print(outputStr)
    
    def run(self, idx):
    
        while True:

            decoded = decodeCommand(self.opcode[idx])
            
            #if len(self.outputList) >= 2 and self.outputList[-2] == -1 and self.outputList[-1] == 0:
            #    print(idx, decoded)

            if decoded[0] == 99:
                print("Finished!")
                return 0
            elif decoded[0] == 1:
                #add p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 + arg2
                #if arg1 + arg2 in [98,59,85]: print(str(arg1) + " + " + str(arg2) + " = " + str(arg1+arg2))
                idx+=4
            elif decoded[0] == 2:
                #multiply p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 * arg2
                #if arg1 * arg2 in [98,59,85]: print(str(arg1) + " * " + str(arg2) + " = " + str(arg1*arg2))
                idx+=4
            elif decoded[0] == 3:
                #input p1
                p1 = self.opcode[idx+1]
                arg1 = getWritePosition(self.opcode,decoded[1],p1)
                # break here and wait for updated input from extern
                print("Waiting for input")
                yield 1
                var = self.input
                self.opcode[arg1] = np.int64(var)
                idx+=2
            elif decoded[0] == 4:
                #output p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                #print("Output: ", arg1)
                self.outputList.append(arg1)
                idx+=2
            elif decoded[0] == 5:
                #jump-if-true p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 != 0:
                    idx=arg2
                else:      
                    idx=idx+3
            elif decoded[0] == 6:
                #jump-if-false p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 == 0:
                    idx=arg2
                else:      
                    idx+=3
            elif decoded[0] == 7:
                #less-than p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 < arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 8:
                #equals p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 == arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 9:
                # change relative base p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                self.opcode.relativeBaseIdx += arg1
                idx+=2
            else:
                print("Invalid code ", self.opcode, idx, self.opcode[idx], decoded)
                return

In [58]:
#task1
opcodeMgr=OpCodeManager(opcodeInput.copy())
computer = Computer(opcodeMgr)
for i,step in enumerate(computer.run(0)):
    break
print(computer.outputList)
print(computer.visualize())

def stringToArray(visString):
    #find length of row
    length = visString.find("\n")
    array = np.array([s for s in visString if s != '\n'])
    print(length,len(array))
    
    return np.reshape(array, (len(array)//length,length) )

array = stringToArray(computer.visualize())
# first dimension: y, second: x
print(array.shape,array[4,0])
sumAlignmentParameters = 0
for i in range(1,array.shape[1]-1):
    for j in range(1,array.shape[0]-1):
        if array[j,i] == '#' and array[j+1,i] == '#' and array[j-1,i] == '#' and array[j,i+1] == '#' and array[j,i-1] == '#':
            # at intersection
            sumAlignmentParameters += i*j
print(sumAlignmentParameters)

Finished!
[46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 35, 35, 35, 35, 35, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 10, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 35, 46, 46, 46, 35, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 10, 46, 46, 46, 46, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 46, 35, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 10, 46, 46, 46, 46, 35, 46, 46, 46, 46, 46, 46, 46, 46, 46, 35, 46, 35, 46, 35, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 10, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 46, 35, 35, 35, 35, 35, 35, 35, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 4

In [37]:
def orientationToPositionOffset(orientation):
    if orientation == 1:
        return np.array([-1,0])
    elif orientation == 2:
        return np.array([0,1])
    elif orientation == 3:
        return np.array([1,0])
    elif orientation == 4: 
        return np.array([0,-1])
    else:
        import sys
        sys.exit("Unknown orientation given! "+str(orientation))
        return
    
def arrayToList(array):
    lis = []
    for i in range(0,array.shape[1]):
        for j in range(0,array.shape[0]):
            if array[j,i] == '#' or array[j,i] == '^':
                lis.append((j,i))   
    return lis
 
def getNewOrientations(orientation):
    if orientation == 1:
        return [2,4]
    elif orientation == 2:
        return [1,3]
    elif orientation == 3:
        return [2,4]
    elif orientation == 4: 
        return [1,3]
    else:
        import sys
        sys.exit("Unknown orientation given! "+str(orientation))
        return
    

def isOnScaffolding(position,lis):
    if tuple(position) in lis:
        return True
    else:
        return False
    
def getDirectionStr(oldDirection,newDirection):
    if newDirection - oldDirection == 1 or (oldDirection == 4 and newDirection == 1):
        return 'R'
    else:
        return 'L'
        
def findPathSequence(lis, startPosition, startOrientation):
    orientation = startOrientation # 1:N, 2:E, 3:S, 4:W
    position=startPosition
    directionSequence=[]
    movementLengthSequence=[]
    
    currentMovementLength=0
    while True:
        newPosition = position + orientationToPositionOffset(orientation)
        #print(position,orientation,directionSequence,movementLengthSequence,newPosition)
        if isOnScaffolding(newPosition,lis):
            currentMovementLength+=1
            position = newPosition
        else:
            if currentMovementLength != 0:
                movementLengthSequence.append(currentMovementLength)
                currentMovementLength = 0
            newPossibleOrientations=getNewOrientations(orientation)
            newOrientation = 0
            if isOnScaffolding(position + orientationToPositionOffset(newPossibleOrientations[0]),lis):
                newOrientation = newPossibleOrientations[0]
            elif isOnScaffolding(position+orientationToPositionOffset(newPossibleOrientations[1]),lis):
                newOrientation = newPossibleOrientations[1]
            else:
                #converged!
                print("Converged!")
                return [directionSequence,movementLengthSequence]
            directionSequence.append(getDirectionStr(orientation,newOrientation))
            orientation = newOrientation
    print(position)
   
lis = arrayToList(array)
path = findPathSequence(lis,np.argwhere(array=='^')[0],1)
print(path)
pathTuples= [i for i in zip(path[0],path[1])]
print(pathTuples)

Converged!
[['L', 'R', 'R', 'R', 'R', 'L', 'R', 'R', 'L', 'L', 'R', 'R', 'R', 'L', 'L', 'L', 'L', 'R', 'R', 'R', 'L', 'L', 'L', 'L', 'R', 'R', 'R', 'L', 'L', 'L', 'R', 'R', 'L'], [6, 12, 8, 8, 12, 12, 8, 12, 12, 6, 12, 8, 12, 12, 4, 4, 6, 12, 8, 12, 12, 4, 4, 6, 12, 8, 12, 12, 4, 4, 8, 12, 12]]
[('L', 6), ('R', 12), ('R', 8), ('R', 8), ('R', 12), ('L', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 6), ('R', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 4), ('L', 4), ('L', 6), ('R', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 4), ('L', 4), ('L', 6), ('R', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 4), ('L', 4), ('R', 8), ('R', 12), ('L', 12)]


In [67]:
# do this manually...
seqA=[('L', 6), ('R', 12), ('R', 8)]
seqB=[('R', 8), ('R', 12), ('L', 12)]
seqC=[('R', 12), ('L', 12), ('L', 4), ('L', 4)]
seqMain=['A','B','B','A','C','A','C','A','C','B']

# check sequence
generatedSeq=[]
for e in seqMain:
    if e == 'A':
        generatedSeq += seqA
    elif e == 'B':
        generatedSeq += seqB
    elif e == 'C':
        generatedSeq += seqC
        
print(generatedSeq)
print(pathTuples)

#translate to ANSII
def seqToANSII(seq): 
    seqANSII=[]
    for i,e in enumerate(seq):
        seqANSII.append(ord(e))
        if i != len(seq)-1:
            seqANSII.append(ord(','))
        else:
            seqANSII.append(ord('\n'))
    return seqANSII

def functionSeqToANSII(seq): 
    seqANSII=[]
    for i,e in enumerate(seq):
        seqANSII.append(ord(e[0]))
        seqANSII.append(ord(','))
        if e[1] > 10:
            seqANSII.append(ord(str(e[1]//10)))
            seqANSII.append(ord(str(e[1]%10)))
        else:
            seqANSII.append(ord(str(e[1])))
        if i != len(seq)-1:
            seqANSII.append(ord(','))
        else:
            seqANSII.append(ord('\n'))
    return seqANSII

seqMainANSII = seqToANSII(seqMain)
seqAANSII = functionSeqToANSII(seqA)
seqBANSII = functionSeqToANSII(seqB)
seqCANSII = functionSeqToANSII(seqC)
print(seqMainANSII)

[('L', 6), ('R', 12), ('R', 8), ('R', 8), ('R', 12), ('L', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 6), ('R', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 4), ('L', 4), ('L', 6), ('R', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 4), ('L', 4), ('L', 6), ('R', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 4), ('L', 4), ('R', 8), ('R', 12), ('L', 12)]
[('L', 6), ('R', 12), ('R', 8), ('R', 8), ('R', 12), ('L', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 6), ('R', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 4), ('L', 4), ('L', 6), ('R', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 4), ('L', 4), ('L', 6), ('R', 12), ('R', 8), ('R', 12), ('L', 12), ('L', 4), ('L', 4), ('R', 8), ('R', 12), ('L', 12)]
[65, 44, 66, 44, 66, 44, 65, 44, 67, 44, 65, 44, 67, 44, 65, 44, 67, 44, 66, 10]


In [72]:
opcodeMgr=OpCodeManager(opcodeInput.copy())
opcodeMgr[0] = 2
computer = Computer(opcodeMgr)

inputSequence=seqMainANSII+seqAANSII+seqBANSII+seqCANSII+[ord('n'),ord('\n')]

print(inputSequence)

for i,step in enumerate(computer.run(0)):
    #print(i,computer.outputList)
    if i == 0:
        print(computer.visualize())
        computer.clearOutput()
    if len(computer.outputList) > 0:
        computer.printOutputList()
        computer.clearOutput()
    if i < len(inputSequence):
        computer.input = inputSequence[i]
print(computer.visualize())
print(computer.outputList[-1])

[65, 44, 66, 44, 66, 44, 65, 44, 67, 44, 65, 44, 67, 44, 65, 44, 67, 44, 66, 10, 76, 44, 54, 44, 82, 44, 49, 50, 44, 82, 44, 56, 10, 82, 44, 56, 44, 82, 44, 49, 50, 44, 76, 44, 49, 50, 10, 82, 44, 49, 50, 44, 76, 44, 49, 50, 44, 76, 44, 52, 44, 76, 44, 52, 10, 110, 10]
Waiting for input
..............#####................................
..............#...#................................
....#############.#................................
....#.........#.#.#................................
#############.#######..............................
#...#.......#...#.#.#..............................
#...#.......#...#.#.#.....................#########
#...#.......#...#.#.#.....................#.......#
#####.......#...#.#.#.....................#.......#
............#...#.#.#.....................#.......#
....#############.#.#.....................#.......#
....#.......#.....#.#.....................#.......#
....#.#############.#.....................#.......#
....#.#.....#.......#...............